In [ ]:
#!/usr/bin/env python

Version:
torch:1.8.1+cu101
torchvision:0.9.1+cu101
torchtext:0.9.1

In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [4]:
# cd G:\マイドライブ\ColabNotebooks\Others\graph-2-text

In [5]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text


In [6]:
# cd /Users/kosukeaigo/Google Drive/ColabNotebooks/Others/graph-2-text

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
from __future__ import division

In [9]:
import argparse
import glob
import os
import sys
import random
from datetime import datetime

In [10]:
import torch
import torch.nn as nn
from torch import cuda

In [11]:
import onmt
import onmt.io
import onmt.Models
import onmt.ModelConstructor
import onmt.modules
from onmt.Utils import use_gpu
import opts

In [12]:
import easydict

In [13]:
parser = argparse.ArgumentParser(
    description='train.py',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)

opts.py

In [14]:
opts.add_md_help_argument(parser)
opts.model_opts(parser)
opts.train_opts(parser)

In [15]:
opts.gcn_opts(parser)  # adds GCN options

In [16]:
opt = parser.parse_args("-data data/gcn_exp20210402 -save_model data/tmp_ -rnn_size 256 -word_vec_size 256 -layers 1 -epochs 10 -optim adam -learning_rate 0.001 -encoder_type gcn -gcn_num_inputs 256 -gcn_num_units 256 -gcn_in_arcs -gcn_out_arcs -gcn_num_layers 1 -gcn_num_labels 5".split())

In [17]:
# opt = parser.parse_args("-gpuid 0 -data data/gcn_exp20210402 -save_model data/tmp_ -rnn_size 256 -word_vec_size 256 -layers 1 -epochs 10 -optim adam -learning_rate 0.001 -encoder_type gcn -gcn_num_inputs 256 -gcn_num_units 256 -gcn_in_arcs -gcn_out_arcs -gcn_num_layers 1 -gcn_num_labels 5".split())

In [18]:
# opt = parser.parse_args("-train_from data/tmp__acc_69.86_ppl_3.36_e9.pt -data data/gcn_exp -save_model data/tmp_ -rnn_size 256 -word_vec_size 256 -layers 1 -epochs 10 -optim adam -learning_rate 0.001 -encoder_type gcn -gcn_num_inputs 256 -gcn_num_units 256 -gcn_in_arcs -gcn_out_arcs -gcn_num_layers 1 -gcn_num_labels 5".split())

In [19]:
# opt = parser.parse_args("-train_from data/tmp__acc_70.54_ppl_3.28_e10.pt -data data/gcn_exp -save_model data/tmp_ -rnn_size 256 -word_vec_size 256 -layers 1 -epochs 10 -optim adam -learning_rate 0.001 -encoder_type gcn -gcn_num_inputs 256 -gcn_num_units 256 -gcn_in_arcs -gcn_out_arcs -gcn_num_layers 1 -gcn_num_labels 5".split())

In [20]:
if opt.word_vec_size != -1:
    opt.src_word_vec_size = opt.word_vec_size
    opt.tgt_word_vec_size = opt.word_vec_size

In [21]:
if opt.layers != -1:
    opt.enc_layers = opt.layers
    opt.dec_layers = opt.layers

In [22]:
opt.brnn = (opt.encoder_type == "brnn")
if opt.seed > 0:
    random.seed(opt.seed)
    torch.manual_seed(opt.seed)

In [23]:
if opt.rnn_type == "SRU" and not opt.gpuid:
    raise AssertionError("Using SRU requires -gpuid set.")

In [24]:
if torch.cuda.is_available() and not opt.gpuid:
    print("WARNING: You have a CUDA device, should run with -gpuid 0")

In [25]:
if opt.gpuid:
    # cuda.set_device(opt.gpuid[0])
    cuda.set_device('cuda:0')
    if opt.seed > 0:
        torch.cuda.manual_seed(opt.seed)

In [26]:
if len(opt.gpuid) > 1:
    sys.stderr.write("Sorry, multigpu isn't supported yet, coming soon!\n")
    sys.exit(1)

Set up the Crayon logging server.

In [27]:
if opt.exp_host != "":
    from pycrayon import CrayonClient
    cc = CrayonClient(hostname=opt.exp_host)
    experiments = cc.get_experiment_names()
    print(experiments)
    if opt.exp in experiments:
        cc.remove_experiment(opt.exp)
    experiment = cc.create_experiment(opt.exp)

In [28]:
if opt.tensorboard:
    from tensorboardX import SummaryWriter
    writer = SummaryWriter(
        opt.tensorboard_log_dir + datetime.now().strftime("/%b-%d_%H-%M-%S"),
        comment="Onmt")

In [29]:
progress_step = 0

In [30]:
def report_func(epoch, batch, num_batches,
                progress_step,
                start_time, lr, report_stats):
    """
    This is the user-defined batch-level traing progress
    report function.
    Args:
        epoch(int): current epoch count.
        batch(int): current batch count.
        num_batches(int): total number of batches.
        progress_step(int): the progress step.
        start_time(float): last report time.
        lr(float): current learning rate.
        report_stats(Statistics): old Statistics instance.
    Returns:
        report_stats(Statistics): updated Statistics instance.
    """
    if batch % opt.report_every == -1 % opt.report_every:
        report_stats.output(epoch, batch + 1, num_batches, start_time)
        if opt.exp_host:
            report_stats.log("progress", experiment, lr)
        if opt.tensorboard:
            # Log the progress using the number of batches on the x-axis.
            report_stats.log_tensorboard(
                "progress", writer, lr, progress_step)
        report_stats = onmt.Statistics()
    return report_stats

In [31]:
class DatasetLazyIter(object):
    """ An Ordered Dataset Iterator, supporting multiple datasets,
        and lazy loading.
    Args:
        datsets (list): a list of datasets, which are lazily loaded.
        fields (dict): fields dict for the datasets.
        batch_size (int): batch size.
        batch_size_fn: custom batch process function.
        device: the GPU device.
        is_train (bool): train or valid?
    """
    def __init__(self, datasets, fields, batch_size, batch_size_fn,
                 device, is_train):
        self.datasets = datasets
        self.fields = fields
        self.batch_size = batch_size
        self.batch_size_fn = batch_size_fn
        self.device = device
        self.is_train = is_train
        self.cur_iter = self._next_dataset_iterator(datasets)
        # We have at least one dataset.
        assert self.cur_iter is not None
    def __iter__(self):
        dataset_iter = (d for d in self.datasets)
        while self.cur_iter is not None:
            for batch in self.cur_iter:
                # print (batch)
                yield batch
            self.cur_iter = self._next_dataset_iterator(dataset_iter)
    def __len__(self):
        # We return the len of cur_dataset, otherwise we need to load
        # all datasets to determine the real len, which loses the benefit
        # of lazy loading.
        assert self.cur_iter is not None
        return len(self.cur_iter)
    def get_cur_dataset(self):
        return self.cur_dataset
    def _next_dataset_iterator(self, dataset_iter):
        try:
            self.cur_dataset = next(dataset_iter)
        except StopIteration:
            return None

        # We clear `fields` when saving, restore when loading.
        self.cur_dataset.fields = self.fields

        # Sort batch by decreasing lengths of sentence required by pytorch.
        # sort=False means "Use dataset's sortkey instead of iterator's".
        return onmt.io.OrderedIterator(
            dataset=self.cur_dataset, batch_size=self.batch_size,
            batch_size_fn=self.batch_size_fn,
            device=self.device, train=self.is_train,
            sort=False, sort_within_batch=True,
            repeat=False)

In [32]:
def make_dataset_iter(datasets, fields, opt, is_train=True):
    """
    This returns user-defined train/validate data iterator for the trainer
    to iterate over during each train epoch. We implement simple
    ordered iterator strategy here, but more sophisticated strategy
    like curriculum learning is ok too.
    """
    batch_size = opt.batch_size if is_train else opt.valid_batch_size
    batch_size_fn = None
    if is_train and opt.batch_type == "tokens":
        global max_src_in_batch, max_tgt_in_batch
        def batch_size_fn(new, count, sofar):
            global max_src_in_batch, max_tgt_in_batch
            if count == 1:
                max_src_in_batch = 0
                max_tgt_in_batch = 0
            max_src_in_batch = max(max_src_in_batch,  len(new.src) + 2)
            max_tgt_in_batch = max(max_tgt_in_batch,  len(new.tgt) + 1)
            src_elements = count * max_src_in_batch
            tgt_elements = count * max_tgt_in_batch
            return max(src_elements, tgt_elements)
    # device = opt.gpuid[0] if opt.gpuid else -1
    device = 'cuda:0' if opt.gpuid else -1
    return DatasetLazyIter(datasets, fields, batch_size, batch_size_fn,
                           device, is_train)

In [33]:
def make_loss_compute(model, tgt_vocab, opt, train=True):
    """
    This returns user-defined LossCompute object, which is used to
    compute loss in train/validate process. You can implement your
    own *LossCompute class, by subclassing LossComputeBase.
    """
    if opt.copy_attn:
        compute = onmt.modules.CopyGeneratorLossCompute(
            model.generator, tgt_vocab, opt.copy_attn_force,
            opt.copy_loss_by_seqlength)
    else:
        compute = onmt.Loss.NMTLossCompute(
            model.generator, tgt_vocab,
            label_smoothing=opt.label_smoothing if train else 0.0)
    if use_gpu(opt):
        compute.cuda()
    return compute

In [34]:
def train_model(model, fields, optim, data_type, model_opt):
    train_loss = make_loss_compute(model, fields["tgt"].vocab, opt)
    valid_loss = make_loss_compute(model, fields["tgt"].vocab, opt,
                                   train=False)
    trunc_size = opt.truncated_decoder  # Badly named...
    shard_size = opt.max_generator_batches
    norm_method = opt.normalization
    grad_accum_count = opt.accum_count
    trainer = onmt.Trainer(model, train_loss, valid_loss, optim,
                           trunc_size, shard_size, data_type,
                           norm_method, grad_accum_count)
    print('\nStart training...')
    print(' * number of epochs: %d, starting from Epoch %d' %
          (opt.epochs + 1 - opt.start_epoch, opt.start_epoch))
    print(' * batch size: %d' % opt.batch_size)
    for epoch in range(opt.start_epoch, opt.epochs + 1):
        print('')

        # 1. Train for one epoch on the training set.
        lld = lazily_load_dataset("train")
        train_iter = make_dataset_iter(lld, fields, opt)
        train_stats = trainer.train(train_iter, epoch, report_func)
        print('Train perplexity: %g' % train_stats.ppl())
        print('Train accuracy: %g' % train_stats.accuracy())

        # 2. Validate on the validation set.
        valid_iter = make_dataset_iter(lazily_load_dataset("valid"),
                                       fields, opt,
                                       is_train=False)
        valid_stats = trainer.validate(valid_iter)
        print('Validation perplexity: %g' % valid_stats.ppl())
        print('Validation accuracy: %g' % valid_stats.accuracy())

        # 3. Log to remote server.
        if opt.exp_host:
            train_stats.log("train", experiment, optim.lr)
            valid_stats.log("valid", experiment, optim.lr)
        if opt.tensorboard:
            train_stats.log_tensorboard("train", writer, optim.lr, epoch)
            train_stats.log_tensorboard("valid", writer, optim.lr, epoch)

        # 4. Update the learning rate
        trainer.epoch_step(valid_stats.ppl(), epoch)

        # 5. Drop a checkpoint if needed.
        if epoch >= opt.start_checkpoint_at:
            trainer.drop_checkpoint(model_opt, epoch, fields, valid_stats)

In [35]:
def check_save_model_path():
    save_model_path = os.path.abspath(opt.save_model)
    model_dirname = os.path.dirname(save_model_path)
    if not os.path.exists(model_dirname):
        os.makedirs(model_dirname)

In [36]:
def tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    print('* number of parameters: %d' % n_params)
    enc = 0
    dec = 0
    for name, param in model.named_parameters():
        if 'encoder' in name:
            enc += param.nelement()
        elif 'decoder' or 'generator' in name:
            dec += param.nelement()
    print('encoder: ', enc)
    print('decoder: ', dec)

In [37]:
def lazily_load_dataset(corpus_type):
    """
    Dataset generator. Don't do extra stuff here, like printing,
    because they will be postponed to the first loading time.
    Args:
        corpus_type: 'train' or 'valid'
    Returns:
        A list of dataset, the dataset(s) are lazily loaded.
    """
    assert corpus_type in ["train", "valid"]
    def lazy_dataset_loader(pt_file, corpus_type):
        dataset = torch.load(pt_file)
        print('Loading %s dataset from %s, number of examples: %d' %
              (corpus_type, pt_file, len(dataset)))
        return dataset

    # Sort the glob output by file name (by increasing indexes).
    pts = sorted(glob.glob(opt.data + '.' + corpus_type + '.[0-9]*.pt'))
    if pts:
        for pt in pts:
            yield lazy_dataset_loader(pt, corpus_type)
    else:
        # Only one onmt.io.*Dataset, simple!
        pt = opt.data + '.' + corpus_type + '.pt'
        yield lazy_dataset_loader(pt, corpus_type)

In [38]:
def load_fields(dataset, data_type, checkpoint):
    if checkpoint is not None:
        print('Loading vocab from checkpoint at %s.' % opt.train_from)
        fields = onmt.io.load_fields_from_vocab(
            checkpoint['vocab'], data_type)
    else:
        fields = onmt.io.load_fields_from_vocab(
            torch.load(opt.data + '.vocab.pt'), data_type)
    # print(dataset.examples[0].__dict__)
    # print([(k, f) for (k, f) in fields.items()])
    fields = dict([(k, f) for (k, f) in fields.items()
                   if k in dataset.examples[0].__dict__])
    if data_type == 'text' or data_type == 'gcn':
        print(' * vocabulary size. source = %d; target = %d' %
              (len(fields['src'].vocab), len(fields['tgt'].vocab)))
    else:
        print(' * vocabulary size. target = %d' %
              (len(fields['tgt'].vocab)))
    return fields

In [39]:
def collect_report_features(fields):
    src_features = onmt.io.collect_features(fields, side='src')
    tgt_features = onmt.io.collect_features(fields, side='tgt')
    for j, feat in enumerate(src_features):
        print(' * src feature %d size = %d' % (j, len(fields[feat].vocab)))
    for j, feat in enumerate(tgt_features):
        print(' * tgt feature %d size = %d' % (j, len(fields[feat].vocab)))

In [40]:
def build_model(model_opt, opt, fields, checkpoint):
    print('Building model...')
    model = onmt.ModelConstructor.make_base_model(model_opt, fields,
                                                  use_gpu(opt), checkpoint)
    if len(opt.gpuid) > 1:
        print('Multi gpu training: ', opt.gpuid)
        # model = nn.DataParallel(model, device_ids=opt.gpuid, dim=1)
        model = nn.DataParallel(model, device_ids='cuda:0', dim=1)
    print(model)
    return model

In [41]:
def build_optim(model, checkpoint):
    if opt.train_from:
        print('Loading optimizer from checkpoint.')
        optim = checkpoint['optim']
        optim.optimizer.load_state_dict(
            checkpoint['optim'].optimizer.state_dict())
    else:
        print('Making optimizer for training.')
        optim = onmt.Optim(
            opt.optim, opt.learning_rate, opt.max_grad_norm,
            lr_decay=opt.learning_rate_decay,
            start_decay_at=opt.start_decay_at,
            beta1=opt.adam_beta1,
            beta2=opt.adam_beta2,
            adagrad_accum=opt.adagrad_accumulator_init,
            decay_method=opt.decay_method,
            warmup_steps=opt.warmup_steps,
            model_size=opt.rnn_size)
    optim.set_parameters(model.named_parameters())
    return optim

In [ ]:
def main():
    # Load checkpoint if we resume from a previous training.
    if opt.train_from:
        print('Loading checkpoint from %s' % opt.train_from)
        checkpoint = torch.load(opt.train_from,
                                map_location=lambda storage, loc: storage)
        model_opt = checkpoint['opt']
        # I don't like reassigning attributes of opt: it's not clear.
        opt.start_epoch = checkpoint['epoch'] + 1
    else:
        checkpoint = None
        model_opt = opt

    # Peek the first dataset to determine the data_type.
    # (All datasets have the same data_type).
    first_dataset = next(lazily_load_dataset("train"))
    data_type = first_dataset.data_type

    # Load fields generated from preprocess phase.
    fields = load_fields(first_dataset, data_type, checkpoint)
    print("-------------------------------TEST----------------------"+str(fields['node1'].dtype)+"------------------------CCOMPLETE")

    # Report src/tgt features.
    collect_report_features(fields)

    # Build model.
    model = build_model(model_opt, opt, fields, checkpoint)  # D: gcn features must be passed here
    tally_parameters(model)
    check_save_model_path()

    # Build optimizer.
    optim = build_optim(model, checkpoint)

    # Do training.
    train_model(model, fields, optim, data_type, model_opt)

    # If using tensorboard for logging, close the writer after training.
    if opt.tensorboard:
        writer.close()

In [ ]:
if __name__ == "__main__":
    main()

Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519
 * vocabulary size. source = 2593; target = 4560
-------------------------------TEST----------------------torch.int64------------------------CCOMPLETE
Building model...
use gates =  False


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:230: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_1)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:232: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_2)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:234: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_3)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_4)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/my_modules/GCN.py:41: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_norma

Intializing model parameters.
NMTModelGCN(
  (encoder): GCNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(2593, 256, padding_idx=1)
        )
      )
    )
    (gcn_seq): Sequential(
      (0): GCNLayer(
        (glu): GLU(dim=3)
        (relu): ReLU()
        (sigmoid): Sigmoid()
      )
    )
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(4560, 256, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(512, 256)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=256, out_features=256, bias=False)
      (linear_out): Linear(in_features=512, out_features=256, bias=False)
      (sm): Softmax(dim=None)
      (tan

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Optim.py:125: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(self.params, self.max_grad_norm)


# One Batch TEST

In [51]:
# Load checkpoint if we resume from a previous training.
if opt.train_from:
    print('Loading checkpoint from %s' % opt.train_from)
    checkpoint = torch.load(opt.train_from,
                            map_location=lambda storage, loc: storage)
    model_opt = checkpoint['opt']
    # I don't like reassigning attributes of opt: it's not clear.
    opt.start_epoch = checkpoint['epoch'] + 1
else:
    checkpoint = None
    model_opt = opt

# Peek the first dataset to determine the data_type.
# (All datasets have the same data_type).
first_dataset = next(lazily_load_dataset("train"))
data_type = first_dataset.data_type

# Load fields generated from preprocess phase.
fields = load_fields(first_dataset, data_type, checkpoint)
print("-------------------------------TEST----------------------"+str(fields['node1'].dtype)+"------------------------CCOMPLETE")

# Report src/tgt features.
collect_report_features(fields)

# Build model.
model = build_model(model_opt, opt, fields, checkpoint)  # D: gcn features must be passed here
tally_parameters(model)
check_save_model_path()

# Build optimizer.
optim = build_optim(model, checkpoint)

# Do training.
# train_model(model, fields, optim, data_type, model_opt)
# def train_model(model, fields, optim, data_type, model_opt):

train_loss = make_loss_compute(model, fields["tgt"].vocab, opt)
valid_loss = make_loss_compute(model, fields["tgt"].vocab, opt,
                                train=False)
trunc_size = opt.truncated_decoder  # Badly named...
shard_size = opt.max_generator_batches
norm_method = opt.normalization
grad_accum_count = opt.accum_count
trainer = onmt.Trainer(model, train_loss, valid_loss, optim,
                        trunc_size, shard_size, data_type,
                        norm_method, grad_accum_count)
print('\nStart training...')
print(' * number of epochs: %d, starting from Epoch %d' %
        (opt.epochs + 1 - opt.start_epoch, opt.start_epoch))
print(' * batch size: %d' % opt.batch_size)
# for epoch in range(opt.start_epoch, opt.epochs + 1):
epoch=0

# 1. Train for one epoch on the training set.
lld = lazily_load_dataset("train")
train_iter = make_dataset_iter(lld, fields, opt)
train_stats = trainer.train(train_iter, epoch, report_func)
print('Train perplexity: %g' % train_stats.ppl())
print('Train accuracy: %g' % train_stats.accuracy())

# 2. Validate on the validation set.
valid_iter = make_dataset_iter(lazily_load_dataset("valid"),
                                fields, opt,
                                is_train=False)
valid_stats = trainer.validate(valid_iter)
print('Validation perplexity: %g' % valid_stats.ppl())
print('Validation accuracy: %g' % valid_stats.accuracy())

# 3. Log to remote server.
if opt.exp_host:
    train_stats.log("train", experiment, optim.lr)
    valid_stats.log("valid", experiment, optim.lr)
if opt.tensorboard:
    train_stats.log_tensorboard("train", writer, optim.lr, epoch)
    train_stats.log_tensorboard("valid", writer, optim.lr, epoch)

# 4. Update the learning rate
trainer.epoch_step(valid_stats.ppl(), epoch)

# 5. Drop a checkpoint if needed.
if epoch >= opt.start_checkpoint_at:
    trainer.drop_checkpoint(model_opt, epoch, fields, valid_stats)

# If using tensorboard for logging, close the writer after training.
if opt.tensorboard:
    writer.close()

Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519
 * vocabulary size. source = 2593; target = 4560
-------------------------------TEST----------------------torch.int64------------------------CCOMPLETE
Building model...
use gates =  False
Intializing model parameters.
NMTModelGCN(
  (encoder): GCNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(2593, 256, padding_idx=1)
        )
      )
    )
    (gcn_seq): Sequential(
      (0): GCNLayer(
        (glu): GLU(dim=3)
        (relu): ReLU()
        (sigmoid): Sigmoid()
      )
    )
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(4560, 256, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): 

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:230: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_1)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:232: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_2)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:234: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_3)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_4)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/my_modules/GCN.py:41: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_norma

Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Optim.py:125: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(self.params, self.max_grad_norm)


Epoch  0,    50/  274; acc:   7.70; ppl: 519.95; xent:   6.25; 838 src tok/s; 1243 tgt tok/s;     58 s elapsed
Epoch  0,   100/  274; acc:  14.65; ppl: 229.29; xent:   5.43; 902 src tok/s; 1278 tgt tok/s;    119 s elapsed
Epoch  0,   150/  274; acc:  21.15; ppl: 101.00; xent:   4.62; 887 src tok/s; 1240 tgt tok/s;    181 s elapsed
Epoch  0,   200/  274; acc:  28.24; ppl:  52.59; xent:   3.96; 772 src tok/s; 1146 tgt tok/s;    245 s elapsed
Epoch  0,   250/  274; acc:  32.70; ppl:  33.00; xent:   3.50; 775 src tok/s; 1107 tgt tok/s;    314 s elapsed


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Train perplexity: 101.358
Train accuracy: 22.2621
Loading valid dataset from data/gcn_exp20210402.valid.1.pt, number of examples: 2163
Validation perplexity: 21.2834
Validation accuracy: 39.11


In [ ]:
lld

<generator object lazily_load_dataset at 0x7f916eaaf450>

In [ ]:
for d in lld:
    print(str(type(d))) 

# Test Functions

## lazily_load_dataset

In [ ]:
def lazily_load_dataset_test(corpus_type):
    """
    Dataset generator. Don't do extra stuff here, like printing,
    because they will be postponed to the first loading time.
    Args:
        corpus_type: 'train' or 'valid'
    Returns:
        A list of dataset, the dataset(s) are lazily loaded.
    """
    assert corpus_type in ["train", "valid"]
    def lazy_dataset_loader(pt_file, corpus_type):
        dataset = torch.load(pt_file)
        print('Loading %s dataset from %s, number of examples: %d' %
              (corpus_type, pt_file, len(dataset)))
        return dataset

    # Sort the glob output by file name (by increasing indexes).
    pts = sorted(glob.glob(opt.data + '.' + corpus_type + '.[0-9]*.pt'))
    # print(pts)
    if pts:
        for pt in pts:
            yield lazy_dataset_loader(pt, corpus_type)
    else:
        # Only one onmt.io.*Dataset, simple!
        pt = opt.data + '.' + corpus_type + '.pt'
        yield lazy_dataset_loader(pt, corpus_type)

In [ ]:
lld = lazily_load_dataset_test("train")

In [ ]:
for i in lld:
    print(i)

Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519


## _gradient_accumulation(self, true_batches, total_stats, report_stats, normalization)

In [ ]:
import time

In [ ]:
class Statistics(object):
    """
    Accumulator for loss statistics.
    Currently calculates:

    * accuracy
    * perplexity
    * elapsed time
    """
    def __init__(self, loss=0, n_words=0, n_correct=0):
        self.loss = loss
        self.n_words = n_words
        self.n_correct = n_correct
        self.n_src_words = 0
        self.start_time = time.time()

    def update(self, stat):
        self.loss += stat.loss
        self.n_words += stat.n_words
        self.n_correct += stat.n_correct

    def accuracy(self):
        return 100 * (self.n_correct / self.n_words)

    def xent(self):
        return self.loss / self.n_words

    def ppl(self):
        return math.exp(min(self.loss / self.n_words, 100))

    def elapsed_time(self):
        return time.time() - self.start_time

    def output(self, epoch, batch, n_batches, start):
        """Write out statistics to stdout.

        Args:
           epoch (int): current epoch
           batch (int): current batch
           n_batch (int): total batches
           start (int): start time of epoch.
        """
        t = self.elapsed_time()
        print(("Epoch %2d, %5d/%5d; acc: %6.2f; ppl: %6.2f; xent: %6.2f; " +
               "%3.0f src tok/s; %3.0f tgt tok/s; %6.0f s elapsed") %
              (epoch, batch,  n_batches,
               self.accuracy(),
               self.ppl(),
               self.xent(),
               self.n_src_words / (t + 1e-5),
               self.n_words / (t + 1e-5),
               time.time() - start))
        sys.stdout.flush()

    def log(self, prefix, experiment, lr):
        t = self.elapsed_time()
        experiment.add_scalar_value(prefix + "_ppl", self.ppl())
        experiment.add_scalar_value(prefix + "_accuracy", self.accuracy())
        experiment.add_scalar_value(prefix + "_tgtper",  self.n_words / t)
        experiment.add_scalar_value(prefix + "_lr", lr)

    def log_tensorboard(self, prefix, writer, lr, step):
        t = self.elapsed_time()
        writer.add_scalar(prefix + "/xent", self.xent(), step)
        writer.add_scalar(prefix + "/ppl", self.ppl(), step)
        writer.add_scalar(prefix + "/accuracy", self.accuracy(), step)
        writer.add_scalar(prefix + "/tgtper",  self.n_words / t, step)
        writer.add_scalar(prefix + "/lr", lr, step)



In [ ]:
def _gradient_accumulation(self, true_batchs, total_stats,
                               report_stats, normalization):
        print("Come in _gradient_accumulation")
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            target_size = batch.tgt.size(0)
            # Truncated BPTT
            if self.trunc_size:
                trunc_size = self.trunc_size
            else:
                trunc_size = target_size

            dec_state = None
            src = onmt.io.make_features(batch, 'src', self.data_type)
            if self.data_type == 'text':
                _, src_lengths = batch.src
                report_stats.n_src_words += src_lengths.sum()
            elif self.data_type == 'gcn':
                _, src_lengths = batch.src
                report_stats.n_src_words += src_lengths.sum()
                adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
                mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
                if hasattr(batch, 'morph'):
                #     doStuff(a.property)
                # except AttributeError:
                #     otherStuff()
                # if batch.morph is not None:
                    morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

            else:
                src_lengths = None

            tgt_outer = onmt.io.make_features(batch, 'tgt')

            for j in range(0, target_size-1, trunc_size):
                # 1. Create truncated target.
                tgt = tgt_outer[j: j + trunc_size]
                # print("tgt",str(tgt))
                # print("size tgt",str(tgt.size()))
                # print("src", str(src))
                # print("size src", str(src.size()))
                
                # for i in tgt.squeeze().view(-1):
                #     print(str(fields["tgt"].vocab.itos[i]), end=' ')

                # 2. F-prop all but generator.
                if self.grad_accum_count == 1:
                    self.model.zero_grad()
                if self.data_type == 'gcn':

                    if hasattr(batch, 'morph'):
                        outputs, attns, dec_state = self.model(src, tgt, src_lengths,
                                   adj_arc_in, adj_arc_out, adj_lab_in,
                                   adj_lab_out, mask_in, mask_out,
                                   mask_loop, mask_sent, morph, mask_morph)
                    else:
                        outputs, attns, dec_state = \
                            self.model(src, tgt, src_lengths,
                                       adj_arc_in, adj_arc_out, adj_lab_in,
                                       adj_lab_out, mask_in, mask_out,
                                       mask_loop, mask_sent)
                else:
                    outputs, attns, dec_state = \
                        self.model(src, tgt, src_lengths, dec_state)

                print("outputs",str(outputs.size()))
                print("attns",str(len(attns)))
                # 3. Compute loss in shards for memory efficiency.
                batch_stats = self.train_loss.sharded_compute_loss(
                        batch, outputs, attns, j,
                        trunc_size, self.shard_size, normalization)

                # 4. Update the parameters and statistics.
                if self.grad_accum_count == 1:
                    self.optim.step()
                total_stats.update(batch_stats)
                report_stats.update(batch_stats)

                # If truncated, don't backprop fully.
                if dec_state is not None:
                    dec_state.detach()

        if self.grad_accum_count > 1:
            self.optim.step()

In [ ]:
def train(self, train_iter, epoch, report_func=None):
        """ Train next epoch.
        Args:
            train_iter: training data iterator
            epoch(int): the epoch number
            report_func(fn): function for logging

        Returns:
            stats (:obj:`onmt.Statistics`): epoch loss statistics
        """
        total_stats = Statistics()
        report_stats = Statistics()
        idx = 0
        true_batchs = []
        accum = 0
        normalization = 0
        try:
            add_on = 0
            if len(train_iter) % self.grad_accum_count > 0:
                add_on += 1
            num_batches = len(train_iter) / self.grad_accum_count + add_on
        except NotImplementedError:
            # Dynamic batching
            num_batches = -1

        # for i, batch in enumerate(train_iter):
        i = 0
        batch = train_iter.__iter__().__next__()
        cur_dataset = train_iter.get_cur_dataset()
        self.train_loss.cur_dataset = cur_dataset

        true_batchs.append(batch)
        accum += 1
        if self.norm_method == "tokens":
            num_tokens = batch.tgt[1:].data.view(-1) \
                .ne(self.train_loss.padding_idx).sum()
            normalization += num_tokens
        else:
            normalization += batch.batch_size

        # print("accum",str(accum.size()))
        # print("grad_accum_count",str(self.grad_accum_count))
        if accum == self.grad_accum_count:
            _gradient_accumulation(
                    trainer,
                    true_batchs, total_stats,
                    report_stats, normalization)

            if report_func is not None:
                report_stats = report_func(
                        epoch, idx, num_batches,
                        self.progress_step,
                        total_stats.start_time, self.optim.lr,
                        report_stats)
                self.progress_step += 1

            true_batchs = []
            accum = 0
            normalization = 0
            idx += 1

        if len(true_batchs) > 0:
            _gradient_accumulation(
                    trainer,
                    true_batchs, total_stats,
                    report_stats, normalization)
            true_batchs = []

        print("grad_accum_count",str(self.grad_accum_count))

        return total_stats

In [ ]:
lld = lazily_load_dataset("train")
train_iter = make_dataset_iter(lld, fields, opt)
train_stats = train(trainer,train_iter, epoch, report_func)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519
Come in _gradient_accumulation


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))


outputs torch.Size([27, 64, 256])
attns 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


grad_accum_count 1


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Optim.py:125: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(self.params, self.max_grad_norm)


## _make_shard_state(self, batch, output, range_, attns=None


In [42]:
import time
from torch.autograd import Variable

In [43]:
class Statistics(object):
    """
    Accumulator for loss statistics.
    Currently calculates:

    * accuracy
    * perplexity
    * elapsed time
    """
    def __init__(self, loss=0, n_words=0, n_correct=0):
        self.loss = loss
        self.n_words = n_words
        self.n_correct = n_correct
        self.n_src_words = 0
        self.start_time = time.time()

    def update(self, stat):
        self.loss += stat.loss
        self.n_words += stat.n_words
        self.n_correct += stat.n_correct

    def accuracy(self):
        return 100 * (self.n_correct / self.n_words)

    def xent(self):
        return self.loss / self.n_words

    def ppl(self):
        return math.exp(min(self.loss / self.n_words, 100))

    def elapsed_time(self):
        return time.time() - self.start_time

    def output(self, epoch, batch, n_batches, start):
        """Write out statistics to stdout.

        Args:
           epoch (int): current epoch
           batch (int): current batch
           n_batch (int): total batches
           start (int): start time of epoch.
        """
        t = self.elapsed_time()
        print(("Epoch %2d, %5d/%5d; acc: %6.2f; ppl: %6.2f; xent: %6.2f; " +
               "%3.0f src tok/s; %3.0f tgt tok/s; %6.0f s elapsed") %
              (epoch, batch,  n_batches,
               self.accuracy(),
               self.ppl(),
               self.xent(),
               self.n_src_words / (t + 1e-5),
               self.n_words / (t + 1e-5),
               time.time() - start))
        sys.stdout.flush()

    def log(self, prefix, experiment, lr):
        t = self.elapsed_time()
        experiment.add_scalar_value(prefix + "_ppl", self.ppl())
        experiment.add_scalar_value(prefix + "_accuracy", self.accuracy())
        experiment.add_scalar_value(prefix + "_tgtper",  self.n_words / t)
        experiment.add_scalar_value(prefix + "_lr", lr)

    def log_tensorboard(self, prefix, writer, lr, step):
        t = self.elapsed_time()
        writer.add_scalar(prefix + "/xent", self.xent(), step)
        writer.add_scalar(prefix + "/ppl", self.ppl(), step)
        writer.add_scalar(prefix + "/accuracy", self.accuracy(), step)
        writer.add_scalar(prefix + "/tgtper",  self.n_words / t, step)
        writer.add_scalar(prefix + "/lr", lr, step)



In [45]:
def train(self, train_iter, epoch, report_func=None):
        """ Train next epoch.
        Args:
            train_iter: training data iterator
            epoch(int): the epoch number
            report_func(fn): function for logging

        Returns:
            stats (:obj:`onmt.Statistics`): epoch loss statistics
        """
        total_stats = Statistics()
        report_stats = Statistics()
        idx = 0
        true_batchs = []
        accum = 0
        normalization = 0
        try:
            add_on = 0
            if len(train_iter) % self.grad_accum_count > 0:
                add_on += 1
            num_batches = len(train_iter) / self.grad_accum_count + add_on
        except NotImplementedError:
            # Dynamic batching
            num_batches = -1

        # for i, batch in enumerate(train_iter):
        i = 0
        batch = train_iter.__iter__().__next__()
        cur_dataset = train_iter.get_cur_dataset()
        self.train_loss.cur_dataset = cur_dataset

        true_batchs.append(batch)
        accum += 1
        if self.norm_method == "tokens":
            num_tokens = batch.tgt[1:].data.view(-1) \
                .ne(self.train_loss.padding_idx).sum()
            normalization += num_tokens
        else:
            normalization += batch.batch_size

        # print("accum",str(accum.size()))
        # print("grad_accum_count",str(self.grad_accum_count))
        if accum == self.grad_accum_count:
            _gradient_accumulation(
                    trainer,
                    true_batchs, total_stats,
                    report_stats, normalization)

            if report_func is not None:
                report_stats = report_func(
                        epoch, idx, num_batches,
                        self.progress_step,
                        total_stats.start_time, self.optim.lr,
                        report_stats)
                self.progress_step += 1

            true_batchs = []
            accum = 0
            normalization = 0
            idx += 1

        if len(true_batchs) > 0:
            _gradient_accumulation(
                    trainer,
                    true_batchs, total_stats,
                    report_stats, normalization)
            true_batchs = []

        print("grad_accum_count",str(self.grad_accum_count))

        return total_stats

In [44]:
def _gradient_accumulation(self, true_batchs, total_stats,
                               report_stats, normalization):
        print("Come in _gradient_accumulation")
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            target_size = batch.tgt.size(0)
            # Truncated BPTT
            if self.trunc_size:
                trunc_size = self.trunc_size
            else:
                trunc_size = target_size

            dec_state = None
            src = onmt.io.make_features(batch, 'src', self.data_type)
            if self.data_type == 'text':
                _, src_lengths = batch.src
                report_stats.n_src_words += src_lengths.sum()
            elif self.data_type == 'gcn':
                _, src_lengths = batch.src
                report_stats.n_src_words += src_lengths.sum()
                adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
                mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
                if hasattr(batch, 'morph'):
                #     doStuff(a.property)
                # except AttributeError:
                #     otherStuff()
                # if batch.morph is not None:
                    morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

            else:
                src_lengths = None

            tgt_outer = onmt.io.make_features(batch, 'tgt')

            for j in range(0, target_size-1, trunc_size):
                # 1. Create truncated target.
                tgt = tgt_outer[j: j + trunc_size]
                # print("tgt",str(tgt))
                # print("size tgt",str(tgt.size()))
                # print("src", str(src))
                # print("size src", str(src.size()))
                
                # for i in tgt.squeeze().view(-1):
                #     print(str(fields["tgt"].vocab.itos[i]), end=' ')

                # 2. F-prop all but generator.
                if self.grad_accum_count == 1:
                    self.model.zero_grad()
                if self.data_type == 'gcn':

                    if hasattr(batch, 'morph'):
                        outputs, attns, dec_state = self.model(src, tgt, src_lengths,
                                   adj_arc_in, adj_arc_out, adj_lab_in,
                                   adj_lab_out, mask_in, mask_out,
                                   mask_loop, mask_sent, morph, mask_morph)
                    else:
                        outputs, attns, dec_state = \
                            self.model(src, tgt, src_lengths,
                                       adj_arc_in, adj_arc_out, adj_lab_in,
                                       adj_lab_out, mask_in, mask_out,
                                       mask_loop, mask_sent)
                else:
                    outputs, attns, dec_state = \
                        self.model(src, tgt, src_lengths, dec_state)

                print("outputs",str(outputs.size()))
                print("attns",str(len(attns)))
                # 3. Compute loss in shards for memory efficiency.
                batch_stats = sharded_compute_loss(
                        self.train_loss,
                        batch, outputs, attns, j,
                        trunc_size, self.shard_size, normalization)

                # 4. Update the parameters and statistics.
                if self.grad_accum_count == 1:
                    self.optim.step()
                total_stats.update(batch_stats)
                report_stats.update(batch_stats)

                # If truncated, don't backprop fully.
                if dec_state is not None:
                    dec_state.detach()

        if self.grad_accum_count > 1:
            self.optim.step()

In [58]:
def sharded_compute_loss(self, batch, output, attns,
                             cur_trunc, trunc_size, shard_size,
                             normalization):
        """Compute the forward loss and backpropagate.  Computation is done
        with shards and optionally truncation for memory efficiency.

        Also supports truncated BPTT for long sequences by taking a
        range in the decoder output sequence to back propagate in.
        Range is from `(cur_trunc, cur_trunc + trunc_size)`.

        Note sharding is an exact efficiency trick to relieve memory
        required for the generation buffers. Truncation is an
        approximate efficiency trick to relieve the memory required
        in the RNN buffers.

        Args:
          batch (batch) : batch of labeled examples
          output (:obj:`FloatTensor`) :
              output of decoder model `[tgt_len x batch x hidden]`
          attns (dict) : dictionary of attention distributions
              `[tgt_len x batch x src_len]`
          cur_trunc (int) : starting position of truncation window
          trunc_size (int) : length of truncation window
          shard_size (int) : maximum number of examples in a shard
          normalization (int) : Loss is divided by this number

        Returns:
            :obj:`onmt.Statistics`: validation loss statistics

        """
        batch_stats = onmt.Statistics()
        range_ = (cur_trunc, cur_trunc + trunc_size)
        shard_state = _make_shard_state(self,batch, output, range_, attns)
        # print("shard_state",shard_state)
        # print("type shard_state",type(shard_state))
        print("size shard_state",str(shard_state["output"].size()))
        print("size shard_state",str(shard_state["target"].size()))

        for shard in shards(shard_state, shard_size):
            loss, stats = self._compute_loss(batch, **shard)
            loss.div(normalization).backward()
            batch_stats.update(stats)

        return batch_stats

In [62]:
def shards(state, shard_size, eval=False):
    """
    Args:
        state: A dictionary which corresponds to the output of
               *LossCompute._make_shard_state(). The values for
               those keys are Tensor-like or None.
        shard_size: The maximum size of the shards yielded by the model.
        eval: If True, only yield the state, nothing else.
              Otherwise, yield shards.

    Yields:
        Each yielded shard is a dict.

    Side effect:
        After the last shard, this function does back-propagation.
    """
    if eval:
        yield filter_shard_state(state, False, True)
    else:
        # non_none: the subdict of the state dictionary where the values
        # are not None.
        non_none = dict(filter_shard_state(state))

        # Now, the iteration:
        # state is a dictionary of sequences of tensor-like but we
        # want a sequence of dictionaries of tensors.
        # First, unzip the dictionary into a sequence of keys and a
        # sequence of tensor-like sequences.
        keys, values = zip(*((k, torch.split(v, shard_size))
                             for k, v in non_none.items()))

        # Now, yield a dictionary for each shard. The keys are always
        # the same. values is a sequence of length #keys where each
        # element is a sequence of length #shards. We want to iterate
        # over the shards, not over the keys: therefore, the values need
        # to be re-zipped by shard and then each shard can be paired
        # with the keys.
        for shard_tensors in zip(*values):
            yield dict(zip(keys, shard_tensors))

        # Assumed backprop'd
        variables = ((state[k], v.grad.data) for k, v in non_none.items()
                     if isinstance(v, Variable) and v.grad is not None)
        inputs, grads = zip(*variables)
        # print("variables",str(variables))
        # print("inputs",str(input))
        # print("grads",str(grads))
        # print("sizeinputs",str(input.size()))
        # print("size grads",str(grads.size()))
        torch.autograd.backward(inputs, grads)

In [63]:
def filter_shard_state(state, requires_grad=True, volatile=False):
    for k, v in state.items():
        if v is not None:
            if isinstance(v, Variable) and v.requires_grad:
                v = Variable(v.data, requires_grad=requires_grad,
                             volatile=volatile)
            yield k, v

In [64]:
def _make_shard_state(self, batch, output, range_, attns=None):
        # print("output",str(output))
        # print("target",str(batch.tgt[range_[0] + 1: range_[1]]))
        # print("size output",str(output.size()))
        # print("size target",str(batch.tgt[range_[0] + 1: range_[1]].size()))
        return {
            "output": output,
            "target": batch.tgt[range_[0] + 1: range_[1]],
        }

In [65]:
lld = lazily_load_dataset("train")
train_iter = make_dataset_iter(lld, fields, opt)
train_stats = train(trainer,train_iter, epoch, report_func)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519
Come in _gradient_accumulation
outputs torch.Size([10, 64, 256])
attns 1
shard_state {'output': tensor([[[ 0.5964,  0.0267,  0.0000,  ...,  1.2438,  0.7697,  0.0000],
         [ 0.6226,  0.4333,  0.3962,  ...,  1.0035,  0.0000,  0.5537],
         [ 1.3083, -1.3806, -1.1801,  ...,  1.0079,  0.4801,  1.2920],
         ...,
         [-0.1433,  0.0000,  0.0000,  ...,  0.8613,  0.8141,  0.9915],
         [-0.0000,  0.0000,  0.3926,  ...,  0.0000,  0.8334,  0.6544],
         [ 0.0000,  0.1359,  0.0000,  ...,  1.1255,  1.1397,  0.7862]],

        [[-1.1075,  1.2259,  0.0000,  ..., -0.0000,  0.0000,  1.2279],
         [-0.0000,  0.0000,  0.3108,  ...,  1.1684, -1.4046,  0.0000],
         [ 0.0000, -1.2138, -0.7043,  ...,  0.0000, -1.2366,  1.3638],
         ...,
         [-0.7616,  0.9746,  0.0000,  ..., -1.2150,  0.5094,  1.2072],
         [-1.4042,  0.8817,  0.9025,  ..., -0.0000,  0.4136,  1.1475],
       

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


variables <generator object shards.<locals>.<genexpr> at 0x7f14df5414d0>
inputs <bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7f1549f07950>>
grads (tensor([[[ 1.5229e-03,  1.2803e-03, -5.7480e-05,  ...,  1.5001e-04,
           2.2770e-03,  1.3792e-03],
         [-1.8571e-03, -1.6060e-03,  4.7606e-04,  ..., -2.8131e-04,
          -6.4832e-04, -5.2858e-04],
         [-3.6798e-06,  1.3627e-03,  6.8531e-04,  ..., -5.9186e-04,
           9.1134e-04,  6.8096e-05],
         ...,
         [ 9.7715e-04,  5.0853e-04, -5.3868e-04,  ..., -9.7668e-04,
          -7.1752e-05, -4.6997e-04],
         [ 1.3150e-03,  1.8686e-04, -1.0969e-03,  ..., -9.1209e-04,
           2.3318e-04, -4.0031e-04],
         [-1.6999e-03, -1.4916e-03,  5.6424e-04,  ..., -1.1588e-04,
          -3.2251e-04, -2.8322e-04]],

        [[-1.3067e-03,  1.1470e-03,  5.2218e-05,  ..., -1.6373e-03,
           1.4927e-04, -8.6346e-04],
         [-2.1061e-03,  7.5448e-04,  9.2949e-04,  ..., -2.3262e-04,
    

AttributeError: ignored

In [ ]:
print("size batch.tgt",str(batch.tgt.t().size()))

In [ ]:
# print("size batch.tgt",str(batch.tgt.t()))
for sents in batch.tgt.t():
    for i in sents:
        print(str(fields["tgt"].vocab.itos[i]),end=' ')
    print("sents",str(sents.size()))

<s> adisham hall is located in haputale , sri lanka . the leader of sri lanka is ranil wickremesinghe and the capital is sri jayawardenepura kotte . </s> sents torch.Size([28])
<s> a fortress of grey ice is from the united states where barack obama is the president and one of the ethnic groups is asian americans . </s> sents torch.Size([28])
<s> antwerp international airport serves antwerp in belgium . philippe of belgium and charles michel are leaders of belgium , where german is the language spoken . </s> sents torch.Size([28])
<s> alcatraz versus the evil librarians is from the united states where one of the ethnic groups is african american and the language used is english . </s> sents torch.Size([28])
<s> the full name of afc ajax is amsterdamsche football club ajax , they played in the 2014 - 15 eredivisie season and have 53502 members . </s> sents torch.Size([28])
<s> the main ingredients of asam pedas are fish cooked in sour and hot sauce . asam pedas is a dish from sumatra , i

# Check fields attributes

In [ ]:
fields

{'indices': <torchtext.legacy.data.field.Field at 0x7f91c62ba890>,
 'label': <torchtext.legacy.data.field.Field at 0x7f91c6290d50>,
 'node1': <torchtext.legacy.data.field.Field at 0x7f91c6290e90>,
 'node2': <torchtext.legacy.data.field.Field at 0x7f9175370a90>,
 'src': <torchtext.legacy.data.field.Field at 0x7f916eb5d0d0>,
 'tgt': <torchtext.legacy.data.field.Field at 0x7f916eb56e90>}

In [ ]:
fields["indices"].unk_token

'<unk>'

In [ ]:
fields["label"].unk_token

'<unk>'

In [ ]:
fields["node1"].unk_token

'<unk>'

In [ ]:
fields["node2"].unk_token

'<unk>'

In [ ]:
fields["src"].unk_token

'<unk>'

In [ ]:
fields["tgt"].unk_token

'<unk>'

# Check TRAIN_ITER TEST

In [ ]:
lld = lazily_load_dataset("train")
train_iter = make_dataset_iter(lld, fields, opt)
batch = train_iter.__iter__().__next__()

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Loading train dataset from data/gcn_exp20210402.train.1.pt, number of examples: 17519


In [ ]:
cur_dataset = train_iter.get_cur_dataset()

In [ ]:
batch


[torchtext.legacy.data.batch.Batch of size 64]
	[.src]:('[torch.LongTensor of size 16x64]', '[torch.LongTensor of size 64]')
	[.tgt]:[torch.LongTensor of size 28x64]
	[.indices]:[torch.LongTensor of size 64]
	[.label]:[torch.LongTensor of size 17x64]
	[.node1]:[torch.LongTensor of size 17x64]
	[.node2]:[torch.LongTensor of size 17x64]

In [ ]:
batch.batch_size

64

In [ ]:
batch.tgt.view(-1)

tensor([2, 2, 2,  ..., 1, 1, 1])

In [ ]:
print(len(batch.tgt.view(-1)))

1792


In [ ]:
transform_batch_tgt = batch.tgt.t()
for i_sents in transform_batch_tgt:
    for i in i_sents:
        print(fields["tgt"].vocab.itos[i],end=' ')
    print('\n')

<s> adisham hall is located in haputale , sri lanka . the leader of sri lanka is ranil wickremesinghe and the capital is sri jayawardenepura kotte . </s> 

<s> a fortress of grey ice is from the united states where barack obama is the president and one of the ethnic groups is asian americans . </s> 

<s> antwerp international airport serves antwerp in belgium . philippe of belgium and charles michel are leaders of belgium , where german is the language spoken . </s> 

<s> alcatraz versus the evil librarians is from the united states where one of the ethnic groups is african american and the language used is english . </s> 

<s> the full name of afc ajax is amsterdamsche football club ajax , they played in the 2014 - 15 eredivisie season and have 53502 members . </s> 

<s> the main ingredients of asam pedas are fish cooked in sour and hot sauce . asam pedas is a dish from sumatra , in indonesia . </s> 

<s> rahm emanuel is a leader in chicago , dupage county , illinois , united states ,

In [ ]:
batch.node1

tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [ 3,  3,  3,  ...,  3,  3,  2],
        [ 4,  4,  4,  ...,  4,  4,  3],
        ...,
        [14, 16, 12,  ..., 16, 14,  2],
        [16,  1,  2,  ...,  1, 16, 14],
        [ 1,  1,  1,  ...,  1,  1, 16]])

In [ ]:
batch.indices

tensor([12352, 11264, 15189, 11284, 10325,  9249, 12247, 16173, 16020, 11297,
         7766, 15574, 13423,  9975,  8650,  7808,  8300, 11026,  6863,  6649,
        10087,  9250,  4578,  8323, 11782,  8647,  8624, 12571,  7183, 11678,
        12796,  5463, 12572, 11285, 14639, 13814,  9928, 12574, 11290,  8457,
         8302,  8866, 10086,  8324,  6967, 11096,  8423,  8308, 11296,  6966,
         9929,  7551, 12021, 12240,  4522, 11023,  4445, 12794,  8307,  8416,
        10889, 11504, 13095, 12973])

In [ ]:
tgt_outer = onmt.io.make_features(batch, 'tgt')

In [ ]:
opt.truncated_decoder

0

In [ ]:
target_size = batch.tgt.size(0)
trunc_size = 1
for j in range(0, target_size-1, trunc_size):
    tgt = tgt_outer[j: j + trunc_size]
    print(tgt)

tensor([[[2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2],
         [2]]])
tensor([[[ 492],
         [  11],
         [ 263],
         [ 304],
         [   5],
         [   5],


In [ ]:
transform_batch_tgt = batch.src[0]
for i_sents in transform_batch_tgt:
    for i in i_sents:
        print(fields["src"].vocab.itos[i],end=' ')
    print('\n')

adisham united belgium united afc asam 300 bandeja ayam united abilene 300 derbyshire atatürk adare afonso buzz acta aek a 1 asam allama buzz ashgabat adare adare ethiopia 1634 angola arem marriott ethiopia united hiv a united ethiopia united 11 buzz asser 1 buzz agremiação administrative 103 buzz united agremiação united administrative buzz 300 alcobendas acta afonso arem buzz 103 a agra bakewell bacon 

country ethnicgroup leadername language numberofmembers country location ingredient region leadername cityserved location leadername designer country cityserved birthplace abbreviation league fullname manager country operatingorganisation was_a_crew_member_of operatingorganisation country architect leadername author location country foundationplace leadername language genus location ethnicgroup leadername language floorcount birthplace location manager was_a_crew_member_of league publisher architect birthplace leadername league ethnicgroup abbreviation was_a_crew_member_of location le

In [ ]:
transform_batch_tgt = batch.node1.t()
for i_sents in transform_batch_tgt:
    for i in i_sents:
        print(fields["src"].vocab.itos[i],end=' ')
    print('\n')

. , country states . of - , location ( ) , , the language a <blank> 

. , country states . of - united location . ) leadername the language a <blank> <blank> 

. , country . airport of - . location ( ) ) the language ) . <blank> 

. , country states . of - united location . ) leadername the language a <blank> <blank> 

. , country . airport of - united location . ) leadername the language a <blank> <blank> 

. , country . airport of . united location ( ) leadername the language a <blank> <blank> 

. , country states , of - , location ( ) leadername , language a <blank> <blank> 

. , country . airport of - . location ( ) , the , a <blank> <blank> 

. , country . airport of - . location ( airport leadername the location a <blank> <blank> 

. , country states . of - united location . ) leadername the language a <blank> <blank> 

. . , country states airport . - united location ( ) . the language a <blank> 

. , country states , of - , location , ) leadername location language a <blank> <b

In [ ]:
print(str(type(batch.tgt[1:].data)))

<class 'torch.Tensor'>


In [ ]:
train_loss = make_loss_compute(model, fields["tgt"].vocab, opt)
batch.tgt[1:].data.ne(train_loss.padding_idx)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]])

In [ ]:
batch.tgt[1:]

tensor([[492,  11, 263,  ...,  91, 215,   5],
        [123, 391,  39,  ...,  13, 658,  47],
        [  6,   8,  18,  ...,   5, 201,  43],
        ...,
        [996,  79,  58,  ...,   1,   1,   1],
        [  4,   4,   4,  ...,   1,   1,   1],
        [  3,   3,   3,  ...,   1,   1,   1]])

In [ ]:
fields["tgt"].vocab.itos[1]

'<blank>'

In [ ]:
batch.tgt[0]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
opt.normalization

'sents'